## 모델 구성
PyTorch 모델은 모델 아키텍처 정의, 손실 함수 지정, 최적화 프로그램 선택 등 여러 주요 단계가 필요하며 일반적으로 torch.nn.Module을 서브클래싱하고 __init__ 및 forward 메서드를 구현하여 정의됩니다.

[ 데이터 준비 ]

데이터셋을 위한 전처리 과정 정의, 로드, 훈련 데이터셋을 훈련 및 검증 세트로 분할

[ 모델 아키텍처 정의 ]

서브클래스 torch.nn.Module
모든 PyTorch 모델은 'torch.nn.Module'을 하위 클래스로 분류해야 합니다. 이 기본 클래스는 매개변수 추적 및 후크 정의와 같은 모든 모델에 대한 공통 기능을 제공합니다.
- __init__에서 레이어 초기화<br>
__init__ 메소드 내에서 모델의 레이어와 매개변수를 정의합니다. PyTorch는 완전히 연결된 레이어를 위한 'nn.Linear', 컨볼루션 레이어를 위한 'nn.Conv2d' 등과 같은 'torch.nn' 모듈에 내장된 레이어의 포괄적인 컬렉션을 제공합니다.
- forward 메소드 구현<br>
forward 방법은 모델이 입력 데이터를 처리하는 방법을 정의합니다. 여기서는 모델의 순방향 전달을 지정하고 필요에 따라 레이어를 연결하고 활성화 기능을 적용합니다.

[ 손실 함수 지정 ]
- 모델을 정의한 후에는 모델의 예측이 목표 데이터와 얼마나 잘 일치하는지 측정하는 손실 함수를 지정해야 합니다. PyTorch는 'torch.nn'에서 분류 작업을 위한 'nn.CrossEntropyLoss', 회귀 작업을 위한 'nn.MSELoss'와 같은 몇 가지 일반적인 손실 함수를 제공합니다.

[ 최적화 프로그램을 선택 ]
- 최적화 프로그램은 손실 함수를 최소화하기 위해 모델 매개변수를 조정합니다. PyTorch의 torch.optim 모듈에는 SGD 및 Adam과 같은 다양한 최적화 알고리즘이 포함되어 있습니다.

[ 모델 훈련 ]
- 훈련에는 데이터세트 반복, 모델 예측, 손실 계산, 모델 매개변수 업데이트가 포함됩니다.

[ 모델 평가 ]
- 훈련 후에는 검증 또는 테스트 세트에서 모델의 성능을 평가하여 모델이 잘 일반화되는지 확인하세요.



Q. mnist 데이터 셋에 대해서 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 수행하고 Bset model을 저장한 후 다시 불러와서 테스트 데이터로 평가한 결과를 출력하세요.

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# MNIST 데이터셋을 위한 전처리 과정 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 컨볼루션 레이어 정의 (입력 채널 1, 출력 채널 20, 커널 크기 5)
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어 정의 (2x2 풀링)
        self.flatten = nn.Flatten()  # 텐서 평탄화
        self.fc1 = nn.Linear(2880, 50)  # 완전 연결 레이어 (입력 크기 2880, 출력 크기 50)
        self.fc2 = nn.Linear(50, 10)  # 출력 레이어 (클래스 수 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # ReLU 활성화 함수 적용 후 맥스풀링
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # ReLU 활성화 함수 적용
        x = self.fc2(x)  # 최종 출력
        return x

model = MyModel()  # 모델 인스턴스 생성

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD 최적화 알고리즘

# 모델 훈련
best_val_loss = float('inf')  # 검증 손실을 추적하기 위한 변수 초기화
patience, trials = 5, 0  # 조기 종료 기준 설정
num_epochs = 20  # 에폭 수 설정
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델을 통한 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        running_loss += loss.item()

    # 검증 단계
    val_loss = 0.0
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader)  # 평균 검증 손실 계산
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f'Validation Loss Decreased({best_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')  # 모델 저장
    else:
        trials += 1
        if trials >= patience:  # 조기 종료 조건 충족 확인
            print("Early stopping triggered")
            break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load('best_model.pth'))  # 모델 상태 불러오기

# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

100%|██████████| 9912422/9912422 [00:00<00:00, 116998669.61it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25411305.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32061209.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18531642.77it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Train Loss: 0.27169100095828375, Val Loss: 0.09337795337543209
Validation Loss Decreased(inf--->0.093378) 	 Saving The Model
Epoch 2, Train Loss: 0.07873471341840922, Val Loss: 0.06651239539248908
Validation Loss Decreased(0.093378--->0.066512) 	 Saving The Model
Epoch 3, Train Loss: 0.05661494212256123, Val Loss: 0.06045699117338642
Validation Loss Decreased(0.066512--->0.060457) 	 Saving The Model
Epoch 4, Train Loss: 0.04346774993495395, Val Loss: 0.05373844565598274
Validation Loss Decreased(0.060457--->0.053738) 	 Saving The Model
Epoch 5, Train Loss: 0.03558639577108746, Val Loss: 0.046705444876700294
Validation Loss Decreased(0.053738--->0.046705) 	 Saving The Model
Epoch 6, Train Loss: 0.02808920132595813, Val Loss: 0.050020811949891254
Epoch 7, Train Loss: 0.02513225558567016, Val Loss: 0.05542768287178912
Epoch 8, Train Loss: 0.01880632781004533, Val Loss: 0.04655337812715864
Validation Loss D

- torch: 이것은 메인 PyTorch 라이브러리입니다. 여기에는 GPU를 통한 가속을 통한 텐서 계산 지원, 신경망 훈련을 용이하게 하는 자동 차별화, 모델 구축 및 훈련을 위한 다양한 유틸리티가 포함됩니다.
- torch.nn: 레이어, 활성화 함수, 손실 함수와 같은 신경망의 구성 요소를 제공하는 PyTorch의 하위 모듈입니다. 신경망의 아키텍처를 정의하는 데 필수적입니다.
- torch.nn.function: 이 모듈에는 torch.nn 레이어에서 사용되는 기능이 포함되어 있습니다. 입력 데이터 및 가중치에 이러한 함수를 직접 사용할 수 있으므로 일부 작업에 더 많은 유연성을 제공합니다. 여기에는 활성화, 손실 계산 및 상태(즉, 가중치)를 유지하지 않는 다양한 기타 작업을 위한 함수가 포함됩니다.
- torch.optim: 이 하위 모듈은 SGD(Stochastic Gradient Descent), Adam 등과 같은 신경망 훈련을 위한 최적화 알고리즘을 제공합니다. 이러한 최적화 프로그램은 계산된 기울기를 기반으로 네트워크의 가중치를 업데이트하는 데 사용됩니다.
- torchvision: 이미지 데이터 작업을 위한 유틸리티를 제공하는 PyTorch 프로젝트의 패키지입니다. 여기에는 사전 정의된 데이터세트(예: MNIST, CIFAR10, FashionMNIST), 모델 아키텍처(예: ResNet, AlexNet) 및 전처리를 위한 일반적인 이미지 변환이 포함됩니다.
- torchvision.transforms: 일반적인 이미지 변환을 제공하는 torchvision 내의 모듈입니다. 이는 이미지를 신경망에 공급하기 전에 데이터 증대 및 이미지 전처리에 사용될 수 있습니다. 예로는 크기 조정, 정규화, 텐서로 변환 등이 있습니다.
- SubsetRandomSampler: 교체 없이 데이터세트에서 요소를 무작위로 샘플링하는 데 사용되는 도구입니다. 데이터 세트를 훈련 및 검증/테스트 세트로 분할하거나 모델 훈련을 위해 사용자 정의 데이터 샘플링 전략을 구현하려는 경우에 특히 유용

Q. PyTorch를 사용하여 FashionMNIST 데이터세트에 대한 분류 모델링 및 평가를 다음과 같은 단계로 수행하세요.
- 1단계: 신경망 모델 정의
- 2단계: FashionMNIST 데이터셋 로드
- 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
- 4단계: 조기 종료를 포함한 모델 학습 및 best model 저장
- 5단계: best model을 로드하고 테스트 데이터셋으로 평가

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# 1단계: 신경망 모델 정의
# Net 클래스는 nn.Module을 상속받아 만들어진 사용자 정의 신경망 모델로, FashionMNIST 데이터셋의 이미지 분류를 위해 설계
class Net(nn.Module):
# 모델의 구조를 정의합니다. 이 모델은 세 개의 완전 연결(fully-connected) 레이어를 포함
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 120)  # 입력으로 28x28 크기의 이미지를 받아 펼친 후(784개의 피처), 120개의 출력 노드로 연결
        self.fc2 = nn.Linear(120, 84) # 120개의 입력 노드에서 84개의 출력 노드로 연결
        self.fc3 = nn.Linear(84, 10)  # 84개의 입력 노드에서 최종적으로 10개의 출력 노드로 연결
# forward 메소드는 신경망에 데이터를 어떻게 전달할지 정의
    def forward(self, x):
        x = x.view(-1, 28 * 28) # 입력 이미지 x를 1차원으로 펼치는 과정입니다. -1은 배치 크기에 대응되며, 이는 어떤 크기의 배치라도 처리할 수 있음을 의미
        x = F.relu(self.fc1(x)) # 첫 번째 레이어를 통과한 후, ReLU 활성화 함수를 적용합니다. 이는 비선형성을 도입하여 모델이 더 복잡한 패턴을 학습
        x = F.relu(self.fc2(x)) # 두 번째 레이어를 통과한 후, 다시 ReLU 활성화 함수를 적용
        x = self.fc3(x) # 세 번째 레이어를 통과한 결과를 반환. 소프트맥스 함수 등을 통해 확률로 변환되어 클래스를 예측하는 데 사용
        return x

# 2단계: FashionMNIST 데이터셋 로드
# torchvision 라이브러리를 사용하여 FashionMNIST 데이터셋을 다운로드하고, 데이터를 전처리하기 위한 변환(transform)을 설정하는 과정

# transforms.Compose는 여러 전처리 단계를 하나로 묶어주는 역할
# transforms.ToTensor(): 이미지를 PyTorch 텐서로 변환. 이미지의 픽셀 값 범위가 0에서 255 사이의 정수에서 0.0에서 1.0 사이의 부동소수점으로 변경
# 모든 채널의 평균을 0.5로, 표준편차를 0.5로 설정합니다. 이는 데이터의 범위를 대략적으로 -1.0에서 1.0 사이로 조정
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# FashionMNIST 데이터셋을 다운로드하고, 지정된 변환을 적용하여 데이터를 준비하는 함수
train_val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 및 검증 분할을 위한 데이터 인덱스 생성
dataset_size = len(train_val_dataset) # 훈련 및 검증 데이터셋의 전체 크기
indices = list(range(dataset_size)) # 데이터셋 내의 모든 샘플에 대한 인덱스를 포함
validation_split = 0.1 # 검증 세트로 사용될 데이터의 비율
split = int(np.floor(validation_split * dataset_size)) # 검증 세트의 크기를 계산
np.random.shuffle(indices) # 훈련 및 검증 세트가 데이터셋의 특정 부분에 치우치지 않도록 하기 위함
train_indices, val_indices = indices[split:], indices[:split] # 섞인 인덱스를 사용하여 훈련 세트와 검증 세트의 인덱스를 분할

# PT 데이터 샘플러 및 로더 생성
# 훈련 세트와 검증 세트에 대한 데이터 로더를 설정하고, 테스트 세트에 대한 데이터 로더를 별도로 설정하는 과정입니다.
# 이러한 데이터 로더들은 모델 학습, 검증, 테스트 과정에서 배치 단위로 데이터를 로드하는 데 사용
train_sampler = SubsetRandomSampler(train_indices) # train_indices에 해당하는 훈련 데이터의 인덱스를 무작위로 샘플링하는 샘플러를 생성
val_sampler = SubsetRandomSampler(val_indices) # val_indices에 해당하는 검증 데이터의 인덱스로부터 데이터를 무작위로 샘플링하는 샘플러를 생성
# 배치 크기 4로 로드하는 훈련 데이터 로더를 생성(# 데이터의 무작위 샘플링을 수행)
trainloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=train_sampler)
valloader = torch.utils.data.DataLoader(train_val_dataset, batch_size=4, sampler=val_sampler)
# shuffle=False 인자를 통해 셔플링 없이 순서대로 데이터를 로드. 테스트 과정에서는 데이터의 순서가 결과에 영향을 미치지 않으므로 셔플링을 수행하지 않습니다.
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3단계: 네트워크, 손실 함수, 최적화 알고리즘 초기화
net = Net() #  클래스의 인스턴스를 생성하여 net 변수에 할당
criterion = nn.CrossEntropyLoss() # 멀티클래스 분류 문제에서 널리 사용되는 손실 함수로, 모델의 예측값과 실제 타겟값 사이의 차이를 측정
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 모멘텀은 최적화 과정에서 이전 그래디언트의 방향을 고려, 파라미터 업데이트 시 관성을 부여

# 4단계: 조기 종료를 포함한 모델 학습
patience = 5 # 검증 손실이 개선되지 않을 때, 훈련을 계속 진행하기 전에 기다릴 에폭 수를 의미
patience_counter = 0
best_val_loss = np.Inf # 최고의 검증 손실 값을 무한대로 초기화. 훈련 과정에서 검증 손실이 이전에 기록된 최소 손실보다 낮아지면 업데이트되는 값

# 모델을 에폭 단위로 반복 훈련시키면서, 각 배치의 손실을 계산하고 모델 파라미터를 업데이트하는 기본적인 훈련 과정을 구현
for epoch in range(10):  # 데이터셋을 여러 번 반복
    net.train()  # 모델을 학습 모드로 설정. 이는 모델 내의 특정 레이어(예: 드롭아웃, 배치 정규화 등)가 훈련 시와 평가 시 다르게 동작해야 할 때 필요
    running_loss = 0.0 # 현재 에폭의 총 손실을 계산하기 위해 실행 손실을 0으로 초기화
    for i, data in enumerate(trainloader, 0): # '0'은 열거의 시작 인덱스를 지정
        inputs, labels = data
        optimizer.zero_grad() # 최적화를 수행하기 전에 모델의 그래디언트를 0으로 초기화
        outputs = net(inputs) # 현재 배치의 입력 데이터를 모델에 전달하여 예측값을 계산
        loss = criterion(outputs, labels) # 모델의 예측값과 실제 레이블 간의 손실을 계산
        loss.backward() # 손실 함수의 그래디언트를 역전파합니다. 이 과정에서 모델 파라미터에 대한 손실의 미분값이 계산
        optimizer.step() # 계산된 그래디언트를 사용하여 모델의 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 실행 손실에 누적. 이를 통해 전체 에폭의 평균 손실을 계산

    # 검증 단계
    net.eval()  # 모델을 평가 모드로 설정
    val_loss = 0.0 # 검증 손실을 계산하기 위한 변수를 0으로 초기화
    with torch.no_grad(): # 이 블록 내에서는 그래디언트 계산을 비활성화. 평가 모드에서는 모델을 업데이트하지 않으므로, 그래디언트를 계산할 필요가 없습니다
        for inputs, labels in valloader: # 검증 데이터 로더(valloader)에서 배치 단위로 데이터를 로드하여 반복
            outputs = net(inputs)
            loss = criterion(outputs, labels) # criterion은 손실 함수로, 모델의 성능을 측정하는 기준
            val_loss += loss.item() # 누적된 검증 손실을 검증 데이터 배치의 총 수로 나누어 평균 검증 손실을 계산

    val_loss /= len(valloader)
    print(f'에폭 {epoch + 1}, 훈련 손실: {running_loss / len(trainloader)}, 검증 손실: {val_loss}')

    # 조기 종료 체크
    if val_loss < best_val_loss: # 현재 에폭에서 계산된 검증 손실(val_loss)이 이전에 기록된 최소 검증 손실(best_val_loss)보다 낮은지 확인
        print(f'검증 손실이 감소하였습니다. ({best_val_loss:.6f} 에서 {val_loss:.6f}로). 모델 저장 중...')
        torch.save(net.state_dict(), '/content/drive/MyDrive/Colab Notebooks/kd_project/models/best_model.pth')
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print('조기 종료 발동.')
            break

# 5단계: 최고의 모델을 로드하고 테스트 데이터셋으로 평가
# torch.load 함수는 지정된 경로에서 모델 파라미터를 불러오며, load_state_dict 메서드를 사용하여 이 파라미터를 현재 net 모델에 로드
net.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/kd_project/models/best_model.pth'))
correct = 0 # 정확히 분류된 샘플의 수를 세기 위한 변수
total = 0 # 테스트셋의 전체 샘플 수를 세기 위한 변수
with torch.no_grad():
    for data in testloader: # 테스트 데이터셋을 배치 단위로 순회
        images, labels = data
        outputs = net(images) # 현재 배치의 이미지를 모델에 전달하여 예측값을 계산
        _, predicted = torch.max(outputs.data, 1) # torch.max는 각 예측에 대한 최대값과 그 위치(인덱스)를 반환. 위치만 필요하므로 _를 사용하여 최대값은 무시
        total += labels.size(0) # labels.size(0)는 현재 배치의 크기(샘플 수)
        correct += (predicted == labels).sum().item() # 일치하는 경우의 수를 텐서 형태로 반환하며, .item()으로 이를 파이썬의 스칼라 값으로 변환

print(f'10000개의 테스트 이미지에 대한 네트워크의 정확도: {100 * correct / total} %')



에폭 1, 훈련 손실: 0.5361356424116228, 검증 손실: 0.4147784432672585
검증 손실이 감소하였습니다. (inf 에서 0.414778로). 모델 저장 중...
에폭 2, 훈련 손실: 0.3928052689871213, 검증 손실: 0.36566980215266814
검증 손실이 감소하였습니다. (0.414778 에서 0.365670로). 모델 저장 중...
에폭 3, 훈련 손실: 0.352077518612075, 검증 손실: 0.4250006926593827
에폭 4, 훈련 손실: 0.32682427865016433, 검증 손실: 0.34593992662386486
검증 손실이 감소하였습니다. (0.365670 에서 0.345940로). 모델 저장 중...
에폭 5, 훈련 손실: 0.3079291260770507, 검증 손실: 0.3171539925446511
검증 손실이 감소하였습니다. (0.345940 에서 0.317154로). 모델 저장 중...
에폭 6, 훈련 손실: 0.29179519376307084, 검증 손실: 0.3220596654096153
에폭 7, 훈련 손실: 0.2794181084287468, 검증 손실: 0.31901972460288863
에폭 8, 훈련 손실: 0.26670340754926425, 검증 손실: 0.32035216963128915
에폭 9, 훈련 손실: 0.25744489532434206, 검증 손실: 0.3150109196861916
검증 손실이 감소하였습니다. (0.317154 에서 0.315011로). 모델 저장 중...
에폭 10, 훈련 손실: 0.24640687478282605, 검증 손실: 0.3320030701239684
10000개의 테스트 이미지에 대한 네트워크의 정확도: 87.74 %


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# 가상 데이터 생성 함수
def generate_data(num_samples=1000, seq_length=10):
    vocab_size = 100  # 어휘집 크기
    data = torch.randint(0, vocab_size, (num_samples, seq_length))
    labels = torch.randint(0, 2, (num_samples,))  # 0 또는 1의 레이블
    return data, labels

data, labels = generate_data()

# 텐서 데이터셋 및 데이터 로더 생성
dataset = TensorDataset(data, labels)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# LSTM 모델 클래스 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        lstm_out, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1,:,:]
        return self.fc(hidden)

model = LSTMModel(vocab_size=100, embed_dim=50, hidden_dim=100, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

# 훈련 함수
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze(1)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 평가 함수
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data, labels in data_loader:
            outputs = model(data).squeeze(1)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = torch.round(torch.sigmoid(outputs))
            correct_predictions = (predictions == labels.unsqueeze(1)).float()
            total_accuracy += correct_predictions.sum().item()
    return total_loss / len(data_loader), total_accuracy / len(data_loader.dataset)

# 조기 종료 로직을 포함한 훈련 및 검증 과정
best_val_loss = float('inf')
patience = 3
trials = 0

for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trials += 1
        if trials >= patience:
            print("조기 종료 발생")
            break

# 테스트 데이터로 최고 모델 평가
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch 1, Train Loss: 0.6938, Val Loss: 0.6977, Val Accuracy: 15.0133
Epoch 2, Train Loss: 0.6783, Val Loss: 0.6946, Val Accuracy: 15.2667
Epoch 3, Train Loss: 0.6616, Val Loss: 0.6936, Val Accuracy: 15.3733
Epoch 4, Train Loss: 0.6383, Val Loss: 0.7041, Val Accuracy: 15.3600
Epoch 5, Train Loss: 0.6014, Val Loss: 0.7069, Val Accuracy: 15.3867
Epoch 6, Train Loss: 0.5603, Val Loss: 0.7668, Val Accuracy: 15.0533
조기 종료 발생
Test Loss: 0.6984, Test Accuracy: 15.0400
